# Librerias

In [17]:
import sys
import os
import re

# Añadir la carpeta raíz del proyecto a sys.path
sys.path.append(os.path.abspath("c:/Users/osmarrincon/Documents/capresoca-data-automation"))
#sys.path.append(os.path.abspath("D:\Proyectos Python\capresoca-data-automation"))

from datetime import datetime  
import pandas as pd  
import numpy as np  
from src.file_loader import cargar_maestros_ADRES

# Rutas y constantes
*1. Oficce*

In [18]:
R_Novedades = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS Negado\all-NS-NEG.txt"
R_Ingresos = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\MS\MS Negado\All_MS_NEG.TXT"
R_S3 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\S3\All-S3.txt"
R_MS_EPS025 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-2\EPS025MS0029052025.TXT"
R_MS_EPSC25 = r"C:\Users\osmarrincon\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0029052025.TXT"

# Ruta de Salida

Carpeta = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #6\ACTIVIDAD 11"

## Constantes

In [19]:
Fecha = "01/05/2025"
N_txt = "GREPS0254062025"

*2. Home*

In [20]:
#R_Novedades = r"D:\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\NS\NS Negado\all-NS-NEG.txt"
#R_Ingresos = r"D:\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\MS\MS Negado\All_MS_NEG.TXT"
#R_S3 = r"D:\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Procesos BDUA EPS\S3\All-S3.txt"
#R_MS_EPS025 = r"D:\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Subsidiados\Maestro\MS\2025-2\EPS025MS0026052025.TXT"
#R_MS_EPSC25 = r"D:\OneDrive - uniminuto.edu\Capresoca\AlmostClear\Procesos BDUA\Contributivo\Maestro\2025-2\EPSC25MC0026052025.TXT"

# Ruta de Salida

#Carpeta = r"C:\Users\crist\OneDrive - 891856000_CAPRESOCA E P S\Escritorio\Yesid Rincón Z\informes\2025\CTO135.2025 Informe  #6\ACTIVIDAD 11"

# Carga de Archivos

In [21]:
# Cargar y combinar los maestros
maestro_combinado = cargar_maestros_ADRES(R_MS_EPS025, R_MS_EPSC25)
# Cargar los archivos en dataframes
df_S3 = pd.read_csv(R_S3, sep=',', header=0, dtype=str, encoding='ansi')
df_Novedades = pd.read_csv(R_Novedades, sep=',', header=0, dtype=str, encoding='ansi')
df_Ingresos = pd.read_csv(R_Ingresos, sep=',', header=0, dtype=str, encoding='ansi')

In [22]:
# Crear el nuevo dataframe con las columnas especificadas y valores vacíos
columnas = ["CONSECUTIVO", "TIPO_GLOSA", "Rev_TPS_IDN_ID", "Rev_HST_IDN_NUMERO_IDENTIFICACION", 
            "Rev_AFL_PRIMER_APELLIDO", "Rev_AFL_SEGUNDO_APELLIDO", "Rev_AFL_PRIMER_NOMBRE", 
            "Rev_AFL_SEGUNDO_NOMBRE", "Rev_AFL_FECHA_NACIMIENTO", "Rev_TPS_GNR_ID", 
            "Correc_TPS_IDN_ID", "Correc_HST_IDN_NUMERO_IDENTIFICACION", "Correc_AFL_PRIMER_APELLIDO", 
            "Correc_AFL_SEGUNDO_APELLIDO", "Correc_AFL_PRIMER_NOMBRE", "Correc_AFL_SEGUNDO_NOMBRE", 
            "Correc_AFL_FECHA_NACIMIENTO", "Correc_TPS_GNR_ID", "NOMBRE_ANEXO"]

Auditoria_059_169 = pd.DataFrame(columns=columnas, dtype=str)

# Limpiar datos

In [23]:
# Convertir columnas a datetime
df_S3['Fecha_Proceso'] = pd.to_datetime(df_S3['Fecha_Proceso'], format='%d/%m/%Y')
df_Ingresos['Fecha_Proceso'] = pd.to_datetime(df_Ingresos['Fecha_Proceso'], format='%d/%m/%Y')
df_Novedades['Fecha_Proceso'] = pd.to_datetime(df_Novedades['Fecha_Proceso'], format='%d/%m/%Y')
fecha_limite = pd.to_datetime(Fecha, format='%d/%m/%Y')

# Columnas clave y orden final
columnas_base = [
    'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO',
    'AFL_SEGUNDO_APELLIDO', 'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE',
    'AFL_FECHA_NACIMIENTO', 'TPS_GNR_ID', 'GLOSA', 'Nombre_Archivo', 'Fecha_Proceso'
]

# Renombres
renombres = {
    'TPS_IDN_ID': "Correc_TPS_IDN_ID",
    'HST_IDN_NUMERO_IDENTIFICACION': "Correc_HST_IDN_NUMERO_IDENTIFICACION",
    'AFL_PRIMER_APELLIDO': "Correc_AFL_PRIMER_APELLIDO",
    'AFL_SEGUNDO_APELLIDO': "Correc_AFL_SEGUNDO_APELLIDO",
    'AFL_PRIMER_NOMBRE': "Correc_AFL_PRIMER_NOMBRE",
    'AFL_SEGUNDO_NOMBRE': "Correc_AFL_SEGUNDO_NOMBRE",
    'AFL_FECHA_NACIMIENTO': "Correc_AFL_FECHA_NACIMIENTO",
    'TPS_GNR_ID': "Correc_TPS_GNR_ID"
}

# -----------------------
# Procesar df_S3
# -----------------------
filtro_S3 = (df_S3['Fecha_Proceso'] >= fecha_limite) & (
    df_S3['GLOSA'].str.contains('GN0059') | df_S3['GLOSA'].str.contains('GN0169')
)
df_S3_filtrado = df_S3.loc[filtro_S3, columnas_base].rename(columns=renombres)

# -----------------------
# Procesar df_Ingresos
# -----------------------
filtro_Ingresos = (df_Ingresos['Fecha_Proceso'] >= fecha_limite) & (
    df_Ingresos['GLOSA'].str.contains('GN0059') | df_Ingresos['GLOSA'].str.contains('GN0169')
)
df_Ingresos_filtrado = df_Ingresos.loc[filtro_Ingresos, columnas_base].rename(columns=renombres)

# -----------------------
# Procesar df_Novedades (sin TPS_GNR_ID)
# -----------------------
filtro_Novedades = (df_Novedades['Fecha_Proceso'] >= fecha_limite) & (
    df_Novedades['GLOSA'].str.contains('GN0059') | df_Novedades['GLOSA'].str.contains('GN0169')
)

# Añadir columna vacía TPS_GNR_ID
df_Novedades_tmp = df_Novedades.copy()
df_Novedades_tmp['TPS_GNR_ID'] = np.nan

df_Novedades_filtrado = df_Novedades_tmp.loc[filtro_Novedades, columnas_base].rename(columns=renombres)

# -----------------------
# Concatenar todo
# -----------------------
Auditoria_059_169 = pd.concat([
    Auditoria_059_169,
    df_S3_filtrado,
    df_Ingresos_filtrado,
    df_Novedades_filtrado
], ignore_index=True)

In [24]:
# Filtrar solo las glosas GN0059 y GN0169 completas, con o sin contenido entre paréntesis
Auditoria_059_169['GLOSA'] = Auditoria_059_169['GLOSA'].apply(
    lambda x: ';'.join(re.findall(r'GN0059(?:\([^)]*\))?|GN0169(?:\([^)]*\))?', x))
)

In [25]:
print(Auditoria_059_169.shape[0])
Auditoria_059_169 = Auditoria_059_169.drop_duplicates(subset=["Correc_TPS_IDN_ID", "Correc_HST_IDN_NUMERO_IDENTIFICACION"])
print(Auditoria_059_169.shape[0])

84
46


In [26]:
# Convertir la variable Fecha a un rango de 3 meses atrás
fecha_inicio = pd.to_datetime(Fecha, format='%d/%m/%Y') - pd.DateOffset(months=3)
fecha_fin = pd.to_datetime(Fecha, format='%d/%m/%Y')

# Filtrar los dataframes por las glosas "GN0059" y "GN0169" y el rango de fechas
glosas_validas = ['GN0059', 'GN0169']

# Filtrar df_Ingresos
df_Ingresos_filtrado = df_Ingresos[
    (df_Ingresos['Fecha_Proceso'] >= fecha_inicio) &
    (df_Ingresos['Fecha_Proceso'] <= fecha_fin) &
    (df_Ingresos['GLOSA'].str.contains('|'.join(glosas_validas)))
]

# Filtrar df_Novedades
df_Novedades_filtrado = df_Novedades[
    (df_Novedades['Fecha_Proceso'] >= fecha_inicio) &
    (df_Novedades['Fecha_Proceso'] <= fecha_fin) &
    (df_Novedades['GLOSA'].str.contains('|'.join(glosas_validas)))
]

# Filtrar df_S3
df_S3_filtrado = df_S3[
    (df_S3['Fecha_Proceso'] >= fecha_inicio) &
    (df_S3['Fecha_Proceso'] <= fecha_fin) &
    (df_S3['GLOSA'].str.contains('|'.join(glosas_validas)))
]

# Función para contar ocurrencias en un dataframe
def contar_ocurrencias(df, col_id, col_hst, auditoria):
    return auditoria.apply(
        lambda row: df[
            (df[col_id] == row['Correc_TPS_IDN_ID']) &
            (df[col_hst] == row['Correc_HST_IDN_NUMERO_IDENTIFICACION'])
        ].shape[0],
        axis=1
    )

# Contar ocurrencias en cada dataframe
Auditoria_059_169['df_Ingresos'] = contar_ocurrencias(
    df_Ingresos_filtrado, 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', Auditoria_059_169
)
Auditoria_059_169['df_Novedades'] = contar_ocurrencias(
    df_Novedades_filtrado, 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', Auditoria_059_169
)
Auditoria_059_169['df_S3'] = contar_ocurrencias(
    df_S3_filtrado, 'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', Auditoria_059_169
)

# Calcular el total de ocurrencias
Auditoria_059_169['Total'] = (
    Auditoria_059_169['df_Ingresos'] +
    Auditoria_059_169['df_Novedades'] +
    Auditoria_059_169['df_S3']
)

In [27]:
# Asegurar que todas las columnas clave sean de tipo string y estén limpias (sin espacios)
for col in ['Correc_TPS_IDN_ID', 'Correc_HST_IDN_NUMERO_IDENTIFICACION']:
    Auditoria_059_169[col] = Auditoria_059_169[col].astype(str).str.strip()

for col in ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION']:
    maestro_combinado[col] = maestro_combinado[col].astype(str).str.strip()

# Realizar merge para traer TPS_GNR_ID desde maestro_combinado
Auditoria_059_169_actualizado = Auditoria_059_169.merge(
    maestro_combinado[['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'TPS_GNR_ID']],
    how='left',
    left_on=['Correc_TPS_IDN_ID', 'Correc_HST_IDN_NUMERO_IDENTIFICACION'],
    right_on=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION']
)

# Actualizar valores vacíos en Correc_TPS_GNR_ID con los traídos del merge
Auditoria_059_169_actualizado['Correc_TPS_GNR_ID'] = Auditoria_059_169_actualizado['Correc_TPS_GNR_ID'].fillna(
    Auditoria_059_169_actualizado['TPS_GNR_ID']
)

# Eliminar columnas auxiliares usadas solo para el merge si no se necesitan
Auditoria_059_169_actualizado.drop(columns=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'TPS_GNR_ID'], inplace=True)

# Asignar el DataFrame actualizado al original
Auditoria_059_169 = Auditoria_059_169_actualizado

# Diagnóstico opcional: mostrar cuántos registros aún tienen Correc_TPS_GNR_ID vacío
faltantes = Auditoria_059_169[Auditoria_059_169['Correc_TPS_GNR_ID'].isna()]
print(f"Registros sin TPS_GNR_ID después del merge: {len(faltantes)}")

Registros sin TPS_GNR_ID después del merge: 1


## TIPO_GLOSA

In [28]:
def tipo_glosa(glosa):
    tiene_59 = 'GN0059' in glosa
    tiene_169 = 'GN0169' in glosa
    if tiene_59 and tiene_169:
        return 3
    elif tiene_59:
        return 1
    elif tiene_169:
        return 2
    else:
        return None

Auditoria_059_169["TIPO_GLOSA"] = Auditoria_059_169["GLOSA"].apply(tipo_glosa)

## Columnas de ADRES

In [29]:
print(Auditoria_059_169["Total"].value_counts())

Total
0     28
8      7
4      3
5      2
7      2
3      1
10     1
2      1
1      1
Name: count, dtype: int64


In [30]:
Auditoria_059_169.shape[0]
Auditoria_059_169.columns
Auditoria_059_169["GLOSA"]

0     GN0169(PRIMER APELLIDO|OLIVARES|OLIVARES MANCO...
1            GN0169(PRIMER APELLIDO|PUERTA|PUERTAS||||)
2     GN0169(|SEGUNDO APELLIDO|CASTELLANOS|CASTELLAN...
3     GN0169(|SEGUNDO APELLIDO|LEGUIZAMON|LEGIZAMON|||)
4                GN0169(||PRIMER NOMBRE|RONALD|RONAL||)
5                                  GN0169(||||SEXO|F|M)
6                                  GN0169(||||SEXO|F|M)
7                                  GN0169(||||SEXO|F|M)
8                                  GN0169(||||SEXO|M|F)
9                GN0169(||PRIMER NOMBRE|ARIANA|ARANA||)
10                                 GN0169(||||SEXO|F|M)
11    GN0169(PRIMER APELLIDO|HINOJOSA|DELGADO|SEGUND...
12                                 GN0169(||||SEXO|F|M)
13                                 GN0169(||||SEXO|F|M)
14          GN0169(|SEGUNDO APELLIDO|ANGULO|AGUDELO|||)
15                GN0169(||PRIMER NOMBRE|KARIM|KARIN||)
16                                       GN0169(||||||)
17                                       GN0169(

In [31]:
# Asegurar que las columnas clave sean de tipo string y estén limpias
for col in ['Correc_TPS_IDN_ID', 'Correc_HST_IDN_NUMERO_IDENTIFICACION']:
    Auditoria_059_169[col] = Auditoria_059_169[col].astype(str).str.strip()

for col in ['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION']:
    maestro_combinado[col] = maestro_combinado[col].astype(str).str.strip()

# Realizar el merge para traer las columnas adicionales desde maestro_combinado
Auditoria_059_169_actualizado = Auditoria_059_169.merge(
    maestro_combinado[[
        'TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION', 'AFL_PRIMER_APELLIDO', 
        'AFL_SEGUNDO_APELLIDO', 'AFL_PRIMER_NOMBRE', 'AFL_SEGUNDO_NOMBRE', 
        'AFL_FECHA_NACIMIENTO', 'TPS_GNR_ID'
    ]],
    how='left',
    left_on=['Correc_TPS_IDN_ID', 'Correc_HST_IDN_NUMERO_IDENTIFICACION'],
    right_on=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION']
)

# Eliminar columnas auxiliares usadas solo para el merge si no se necesitan
Auditoria_059_169_actualizado.drop(columns=['TPS_IDN_ID', 'HST_IDN_NUMERO_IDENTIFICACION'], inplace=True)

# Asignar el DataFrame actualizado al original
Auditoria_059_169 = Auditoria_059_169_actualizado

# Guardar resultados

In [32]:
import os

# Crear la carpeta si no existe
ruta_carpeta = os.path.join(Carpeta, N_txt)
os.makedirs(ruta_carpeta, exist_ok=True)

# Guardar el DataFrame en un archivo Excel
ruta_excel = os.path.join(Carpeta, f"{N_txt}.xlsx")
Auditoria_059_169.to_excel(ruta_excel, index=False)

# Guardar el DataFrame en un archivo .txt sin las últimas 5 columnas
ruta_txt = os.path.join(Carpeta, f"{N_txt}.txt")
Auditoria_059_169.iloc[:, :-5].to_csv(ruta_txt, sep=',', index=False, header=False, encoding='ansi')